# Convert PyTorch Fall Detection Model to CoreML for iPhone Deployment

This notebook converts the trained PyTorch CNN model to CoreML format for deployment on iPhone devices.

*Note: Conversion from PyTorch to CoreML is not compatible on Windows

## 1. Install Required Packages

Install all necessary libraries for PyTorch and CoreML conversion.

In [11]:
# Install required packages
import sys
import subprocess

def install_packages():
    """Install all required packages for CoreML conversion"""
    
    print("Checking Python version...")
    print(f"Python version: {sys.version}")
    python_version = sys.version_info
    
    packages = [
        'torch',
        'torchvision', 
        'numpy',
        'pandas'
    ]
    
    print("Installing required packages...")
    print("="*80)
    
    for package in packages:
        print(f"\n📦 Installing {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
            print(f"✅ {package} installed successfully")
        except subprocess.CalledProcessError:
            print(f"⚠️  {package} installation failed, it might already be installed")
    
    # Install coremltools separately with special handling
    print(f"\n📦 Installing CoreML Tools...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "coremltools"])
        print(f"✅ CoreML Tools installed successfully")
    except subprocess.CalledProcessError:
        print(f"⚠️  CoreML Tools installation failed, it might already be installed")
    
    print("\n" + "="*80)
    print("✅ Package installation completed!")
    print("\nVerifying installations...")
    
    # Verify installations
    try:
        import torch
        print(f"  ✅ PyTorch: {torch.__version__}")
    except ImportError as e:
        print(f"  ❌ PyTorch import failed: {e}")
    
    try:
        import coremltools as ct
        print(f"  ✅ CoreML Tools: {ct.__version__}")
    except ImportError as e:
        print(f"  ❌ CoreML Tools import failed: {e}")
        print("\n" + "="*80)
        print("TROUBLESHOOTING:")
        print("="*80)
        print("If CoreML Tools fails to import with Python 3.13+:")
        print("1. Consider using Python 3.11 or 3.12 (recommended)")
        print("2. Create a new virtual environment with Python 3.12:")
        print("   python3.12 -m venv .venv")
        print("   source .venv/bin/activate  # On macOS/Linux")
        print("   pip install coremltools torch torchvision numpy pandas")
        print("\nAlternatively, continue with the notebook - some features may work.")
        return False
    
    try:
        import numpy as np
        print(f"  ✅ NumPy: {np.__version__}")
    except ImportError as e:
        print(f"  ❌ NumPy import failed: {e}")
    
    try:
        import pandas as pd
        print(f"  ✅ Pandas: {pd.__version__}")
    except ImportError as e:
        print(f"  ❌ Pandas import failed: {e}")
    
    return True

# Run installation
success = install_packages()

if not success:
    print("\n" + "="*80)
    print("⚠️  INSTALLATION INCOMPLETE")
    print("="*80)
    print("Please resolve the issues above before proceeding.")
else:
    print("\n" + "="*80)
    print("✅ All packages installed and verified successfully!")
    print("="*80)

Checking Python version...
Python version: 3.10.11 (main, Dec  1 2025, 17:48:05) [Clang 15.0.0 (clang-1500.1.0.2.5)]
Installing required packages...

📦 Installing torch...



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


✅ torch installed successfully

📦 Installing torchvision...



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


✅ torchvision installed successfully

📦 Installing numpy...



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


✅ numpy installed successfully

📦 Installing pandas...



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


✅ pandas installed successfully

📦 Installing CoreML Tools...
✅ CoreML Tools installed successfully

✅ Package installation completed!

Verifying installations...
  ✅ PyTorch: 2.9.1
  ✅ CoreML Tools: 9.0
  ✅ NumPy: 2.2.6
  ✅ Pandas: 2.3.3

✅ All packages installed and verified successfully!
✅ CoreML Tools installed successfully

✅ Package installation completed!

Verifying installations...
  ✅ PyTorch: 2.9.1
  ✅ CoreML Tools: 9.0
  ✅ NumPy: 2.2.6
  ✅ Pandas: 2.3.3

✅ All packages installed and verified successfully!



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### Python Version Compatibility Note

This Jupyter Notebook is ran with the Python version 3.10.11. Newer version of Python may not be compatible with `coremltools` or other relevant packages.

## 2. Import Libraries

Import all necessary libraries for model conversion.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import coremltools as ct
import numpy as np
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CoreML Tools version: {ct.__version__}")
print(f"\nReady for model conversion!")

✅ Libraries imported successfully!
PyTorch version: 2.9.1
CoreML Tools version: 9.0

Ready for model conversion!


## 3. Define Model Architecture

Define the exact same ComplexCNN1D architecture used in training.

In [13]:
# Define the ComplexCNN1D model class (must match training architecture exactly)
class ComplexCNN1D(nn.Module):
    def __init__(self, input_shape):
        super(ComplexCNN1D, self).__init__()
        self.input_shape = input_shape

        # Initial Conv Layer
        self.conv1 = nn.Conv1d(in_channels=input_shape[1], out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)

        # First Residual Block
        self.conv2_shortcut = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1, padding=0)
        self.conv2_1 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2_1 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)
        self.conv2_2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.bn2_2 = nn.BatchNorm1d(128)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)

        # Second Residual Block
        self.conv3_shortcut = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=1, padding=0)
        self.conv3_1 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3_1 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.4)
        self.conv3_2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3_2 = nn.BatchNorm1d(256)
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)

        # Third Residual Block
        self.conv4_shortcut = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=1, padding=0)
        self.conv4_1 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn4_1 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(0.5)
        self.conv4_2 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.bn4_2 = nn.BatchNorm1d(512)
        self.maxpool4 = nn.MaxPool1d(kernel_size=2)

        # Global Average Pooling and Fully Connected Layers
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(512, 1024)
        self.dropout5 = nn.Dropout(0.6)
        self.fc2 = nn.Linear(1024, 512)
        self.dropout6 = nn.Dropout(0.6)
        self.fc3 = nn.Linear(512, 1)

    def forward(self, x):
        # Initial Conv Layer
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        # Note: Dropout is disabled during eval mode

        # First Residual Block
        shortcut = self.conv2_shortcut(x)
        x = F.relu(self.conv2_1(x))
        x = self.bn2_1(x)
        x = F.relu(self.conv2_2(x))
        x = self.bn2_2(x)
        x = x + shortcut  # Residual connection
        x = self.maxpool2(x)

        # Second Residual Block
        shortcut = self.conv3_shortcut(x)
        x = F.relu(self.conv3_1(x))
        x = self.bn3_1(x)
        x = F.relu(self.conv3_2(x))
        x = self.bn3_2(x)
        x = x + shortcut  # Residual connection
        x = self.maxpool3(x)

        # Third Residual Block
        shortcut = self.conv4_shortcut(x)
        x = F.relu(self.conv4_1(x))
        x = self.bn4_1(x)
        x = F.relu(self.conv4_2(x))
        x = self.bn4_2(x)
        x = x + shortcut  # Residual connection
        x = self.maxpool4(x)

        # Global Average Pooling and Fully Connected Layers
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid activation for binary classification

        return x

print("✅ Model class defined successfully!")
print("Model architecture: ComplexCNN1D with residual blocks")
print("Input shape: (batch_size, 6, 200)")
print("Output: Fall probability (0-1)")

✅ Model class defined successfully!
Model architecture: ComplexCNN1D with residual blocks
Input shape: (batch_size, 6, 200)
Output: Fall probability (0-1)


## 4. Load Trained PyTorch Model

Load the trained model weights from the .pth file.

In [14]:
# Load the trained PyTorch model
MODEL_PATH = 'right_pocket_model.pth'

try:
    # Initialize model
    model = ComplexCNN1D(input_shape=(200, 6))
    
    # Load trained weights
    model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
    
    # Set to evaluation mode (important for BatchNorm and Dropout)
    model.eval()
    
    print("✅ PyTorch model loaded successfully!")
    print(f"Model file: {MODEL_PATH}")
    print(f"Model mode: Evaluation")
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\nModel Statistics:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Model size: ~{total_params * 4 / (1024**2):.2f} MB (float32)")
    
except FileNotFoundError:
    print(f"❌ Error: {MODEL_PATH} not found!")
    print("Please ensure the trained model file is in the current directory.")
except Exception as e:
    print(f"❌ Error loading model: {e}")

✅ PyTorch model loaded successfully!
Model file: right_pocket_model.pth
Model mode: Evaluation

Model Statistics:
  Total parameters: 2,778,561
  Trainable parameters: 2,778,561
  Model size: ~10.60 MB (float32)


## 5. Test PyTorch Model

Verify the PyTorch model works correctly before conversion.

In [15]:
# Test the PyTorch model with dummy data
print("Testing PyTorch model with dummy input...")
print("="*80)

# Create dummy input: (batch_size=1, channels=6, time_steps=200)
dummy_input = torch.randn(1, 6, 200)

print(f"Dummy input shape: {dummy_input.shape}")
print(f"Input range: [{dummy_input.min().item():.3f}, {dummy_input.max().item():.3f}]")

# Run inference
with torch.no_grad():
    output = model(dummy_input)

print(f"\nOutput shape: {output.shape}")
print(f"Output value (fall probability): {output.item():.6f}")
print(f"Prediction: {'Fall' if output.item() >= 0.5 else 'Normal'}")

print("\n✅ PyTorch model is working correctly!")
print("Ready for CoreML conversion.")

Testing PyTorch model with dummy input...
Dummy input shape: torch.Size([1, 6, 200])
Input range: [-3.761, 3.378]

Output shape: torch.Size([1, 1])
Output value (fall probability): 0.000000
Prediction: Normal

✅ PyTorch model is working correctly!
Ready for CoreML conversion.


## 6. Convert to CoreML Format

Convert the PyTorch model to CoreML format for iOS deployment.

In [16]:
# Convert PyTorch model to CoreML
print("Starting CoreML conversion...")
print("="*80)

# Create example input for tracing
example_input = torch.randn(1, 6, 200)

# Trace the model
print("\n📝 Step 1: Tracing PyTorch model...")
traced_model = torch.jit.trace(model, example_input)
print("✅ Model traced successfully!")

# Convert to CoreML
print("\n🔄 Step 2: Converting to CoreML format...")
print("This may take a few moments...")

try:
    # Convert with metadata
    coreml_model = ct.convert(
        traced_model,
        inputs=[ct.TensorType(
            name="sensor_data",
            shape=(1, 6, 200),
            dtype=np.float32
        )],
        outputs=[ct.TensorType(name="fall_probability")],
        minimum_deployment_target=ct.target.iOS15,
        convert_to="mlprogram"
    )
    
    print("✅ CoreML conversion successful!")
    
    # Add model metadata
    print("\n📋 Step 3: Adding model metadata...")
    coreml_model.author = "Fall Detection System"
    coreml_model.license = "Research Use"
    coreml_model.short_description = "CNN model for fall detection using smartphone sensor data"
    coreml_model.version = "1.0"
    
    # Input description
    coreml_model.input_description["sensor_data"] = (
        "6-channel sensor data (ax, ay, az, gx, gy, gz) with 200 time steps. "
        "Shape: (1, 6, 200). Accelerometer in m/s^2, gyroscope in deg/s."
    )
    
    # Output description
    coreml_model.output_description["fall_probability"] = (
        "Probability of fall detection (0-1). Values >= 0.5 indicate fall detected."
    )
    
    print("✅ Metadata added successfully!")
    
    # Display model information
    print("\n" + "="*80)
    print("COREML MODEL INFORMATION")
    print("="*80)
    print(f"\nModel Type: {type(coreml_model).__name__}")
    print(f"iOS Deployment Target: iOS 15+")
    print(f"\nInput:")
    print(f"  Name: sensor_data")
    print(f"  Shape: (1, 6, 200)")
    print(f"  Type: Float32")
    print(f"  Description: 6-channel sensor data (ax, ay, az, gx, gy, gz)")
    print(f"\nOutput:")
    print(f"  Name: fall_probability")
    print(f"  Type: Float32")
    print(f"  Description: Fall detection probability (0-1)")
    
except Exception as e:
    print(f"❌ Error during conversion: {e}")
    print("\nTroubleshooting tips:")
    print("  1. Ensure coremltools is up to date: pip install -U coremltools")
    print("  2. Check that PyTorch model is in eval mode")
    print("  3. Verify input shape matches model expectations")

Starting CoreML conversion...

📝 Step 1: Tracing PyTorch model...
✅ Model traced successfully!

🔄 Step 2: Converting to CoreML format...
This may take a few moments...
✅ Model traced successfully!

🔄 Step 2: Converting to CoreML format...
This may take a few moments...


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 409.24 passes/s]



✅ CoreML conversion successful!

📋 Step 3: Adding model metadata...
✅ Metadata added successfully!

COREML MODEL INFORMATION

Model Type: MLModel
iOS Deployment Target: iOS 15+

Input:
  Name: sensor_data
  Shape: (1, 6, 200)
  Type: Float32
  Description: 6-channel sensor data (ax, ay, az, gx, gy, gz)

Output:
  Name: fall_probability
  Type: Float32
  Description: Fall detection probability (0-1)


## 7. Test CoreML Model

Verify the CoreML model produces the same results as the PyTorch model.

In [17]:
# Test CoreML model and compare with PyTorch
print("Testing CoreML model...")
print("="*80)

# Create test input
test_input = torch.randn(1, 6, 200)
test_input_np = test_input.numpy()

# PyTorch prediction
print("\n🔵 PyTorch Model Prediction:")
with torch.no_grad():
    pytorch_output = model(test_input)
    pytorch_prob = pytorch_output.item()

print(f"  Probability: {pytorch_prob:.6f}")
print(f"  Prediction: {'Fall' if pytorch_prob >= 0.5 else 'Normal'}")

# CoreML prediction
print("\n🍎 CoreML Model Prediction:")
coreml_input = {"sensor_data": test_input_np}
coreml_output = coreml_model.predict(coreml_input)
coreml_prob = float(coreml_output["fall_probability"][0])

print(f"  Probability: {coreml_prob:.6f}")
print(f"  Prediction: {'Fall' if coreml_prob >= 0.5 else 'Normal'}")

# Compare results
print("\n" + "="*80)
print("COMPARISON RESULTS")
print("="*80)
difference = abs(pytorch_prob - coreml_prob)
print(f"\nAbsolute difference: {difference:.8f}")

if difference < 1e-5:
    print("✅ EXCELLENT: Models match perfectly!")
elif difference < 1e-3:
    print("✅ GOOD: Models match within acceptable tolerance")
else:
    print("⚠️  WARNING: Models have significant difference")
    print("This may be due to different precision or optimization")

print("\n✅ CoreML model validation complete!")

Testing CoreML model...

🔵 PyTorch Model Prediction:
  Probability: 0.000000
  Prediction: Normal

🍎 CoreML Model Prediction:
  Probability: 0.000000
  Prediction: Normal

COMPARISON RESULTS

Absolute difference: 0.00000015
✅ EXCELLENT: Models match perfectly!

✅ CoreML model validation complete!


## 8. Save CoreML Model

Save the converted model to a .mlpackage file for iOS deployment.

In [18]:
# Save CoreML model
OUTPUT_PATH = "FallDetectionModel.mlpackage"

print(f"Saving CoreML model to: {OUTPUT_PATH}")
print("="*80)

try:
    coreml_model.save(OUTPUT_PATH)
    
    print(f"\n✅ CoreML model saved successfully!")
    print(f"\nOutput file: {OUTPUT_PATH}")
    print(f"File type: ML Package (iOS 15+)")
    
    # Get file size
    import os
    if os.path.exists(OUTPUT_PATH):
        size_mb = sum(os.path.getsize(os.path.join(dirpath, filename)) 
                     for dirpath, dirnames, filenames in os.walk(OUTPUT_PATH) 
                     for filename in filenames) / (1024 * 1024)
        print(f"Model size: ~{size_mb:.2f} MB")
    
    print("\n" + "="*80)
    print("MODEL DEPLOYMENT READY!")
    print("="*80)
    print("\nNext steps for iOS deployment:")
    print("1. Import the .mlpackage file into Xcode project")
    print("2. Xcode will automatically generate Swift model code")
    print("3. Use the model in the iOS app with Core ML framework")
    
except Exception as e:
    print(f"❌ Error saving model: {e}")

Saving CoreML model to: FallDetectionModel.mlpackage

✅ CoreML model saved successfully!

Output file: FallDetectionModel.mlpackage
File type: ML Package (iOS 15+)
Model size: ~5.34 MB

MODEL DEPLOYMENT READY!

Next steps for iOS deployment:
1. Import the .mlpackage file into Xcode project
2. Xcode will automatically generate Swift model code
3. Use the model in the iOS app with Core ML framework
